In [ ]:
import torch
import sys
import os
import numpy as np
import pandas
from pathlib import Path
import music21 as m21

sys.path.append(os.path.dirname(os.getcwd()))
from src.models.inference import single_piece_predict
from src.models.models import PKSpell
from src.models.process_score import process_score

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using", device)

torch.serialization.add_safe_globals([PKSpell])
torch.serialization.add_safe_globals([torch.nn.modules.rnn.GRU])
torch.serialization.add_safe_globals([torch.nn.modules.linear.Linear])
torch.serialization.add_safe_globals([torch.nn.modules.loss.CrossEntropyLoss])

model = torch.load(Path("../models/pkspell.pt"))

# process_score("tests/test_scores/bach_bwv867P_wrong.xml", "tests/test_scores/pkspelled_testscore.xml", model, device)

def extract1(directory_path, score_suffix):
    """return the first file with given suffix found in the given directory path"""
    files = os.listdir(directory_path)
    for file in files:    
        file_ext = os.path.splitext(file)[1]   # get the extension in the file name
        if (file_ext in score_suffix):
            return directory_path/file
    return # not found
    
def get_corpus(dataset_path, flat=True):
    """build a dictionary of XML scores with corresponding paths"""
    """flat: all XML files are contained in the given path or in subdirs"""
    # default score file name
    score_suffix = ['.musicxml', '.xml', '.mxml']
    if not os.path.exists(dataset_path):
        print('Error: ', dataset_path, 'not found')
        return
    # map: opus_name -> path
    dataset = dict()
    files = os.listdir(dataset_path)
    for file in files:    
        filepath = dataset_path/file
        # skip directories
        if not flat and os.path.isdir(filepath):
            target = extract1(filepath, score_suffix)
            if target is not None:
                dataset[file] = target
        # check the extension in the file name
        elif flat and (os.path.splitext(file)[1] in score_suffix):
            # map score name to file path
            dataset[os.path.splitext(file)[0]] = filepath
    # sort the list alphabetically
    dataset = dict(sorted(dataset.items()))
    return dataset
    
def extract_measure(m, notes, csflag=9):
	for e in m: 
		# case of single note
		if isinstance(e, m21.note.Note):
			notes.append(e)
		# case of chord symbol in jazz harmony (descendant of Chord)
		elif isinstance(e, m21.chord.Chord) and \
				(csflag != 0 or not isinstance(e, m21.harmony.ChordSymbol)):
			for cn in e:
				assert(isinstance(cn, m21.note.Note))
				notes.append(cn)

def extract_part(part, csflag=9):
	notes = []
	mes = part.getElementsByClass(m21.stream.Measure)
	for m in mes:			
		extract_measure(m, notes, csflag)
	return notes

def extract_score(score, csflag=9):
	"""extract the notes to be spelled from a score"""
	"""csflag=0: add the same notes as for PSE evaluation, ignoring Chord Symbols"""
	"""csflag=1: add the same notes as for PSE evaluation, spelling notes of Chord Symbols"""
	"""other csflag: PKspell standard option"""
	if csflag in [0, 1]: # analogous to PSE eval
		lp = score.getElementsByClass(m21.stream.Part)
		assert(len(lp) > 0)
		if (len(lp) > 1):
			print(len(lp), 'parts', 'spelling only the first')
		return extract_part(lp[0], csflag)		
	else: # PKspell defaults
		return score.flat.notes

def diff_list(notes, spelled):
	"""diff list computed under same condition as in PSE eval"""
	assert(len(notes) == len(spelled[0]))
	dl = []
	for i in range(len(notes)):
		skip_tie = (notes[i].tie is not None) and (notes[i].tie.type != 'start')
		if not skip_tie and (notes[i].pitches[0].step != m21.pitch.Pitch(spelled[0][i]).step):
			dl.append(i)
	return dl

def eval_score(score, csflag=9):
	notes = extract_score(score, csflag)
	# pitch class of each note (or chord symbol)
	pcs = [p.midi % 12 for n in notes for p in n.pitches]
	# duration of each note (or chord symbol)
	durs = [n.duration.quarterLength for n in notes for p in n.pitches]
	assert(len(pcs) == len(durs))
	ks_gt = score.getElementsByClass(m21.stream.Part)[0].getElementsByClass(m21.stream.Measure)[0].getElementsByClass([m21.key.Key, m21.key.KeySignature])[0].sharps
      # spell with PKspell
	spelled = single_piece_predict(pcs, durs, model, device)
	ks = spelled[1][0]
	assert(len(spelled[0]) == len(notes))
	#res = list(zip(notes, spelled[0]))
	#dl = [(res[i][0], res[i][1]) for i in range(len(res)) if res[i][0].pitches[0].step != m21.pitch.Pitch(res[i][1]).step]
	dl = diff_list(notes, spelled)
	return(ks_gt,  ks, len(pcs), len(dl))

def eval_opus(dataset, name, csflag=9):
	assert(len(name) > 0)
	if (dataset.get(name) == None):
		print(name, "not found in dataset")
		return
	file = dataset[name]
	score = m21.converter.parse(file.as_posix())
	return eval_score(score, csflag)

def eval_corpus(dataset, skip = [], csflag=9):
	names = sorted(list(dataset)) 
	table = []
	for name in names:
		if (name in skip):
			print('\n', name, 'SKIP\n', flush=True)
			continue
		print('\n evaluation of', name)
		(ks_gt,  ks, nn, err) = eval_opus(dataset, name, csflag)
		print('estimated KS:', ks, 'ground truth:', ks_gt)
		assert(nn > 0)
		pc = (nn - err) * 100 / nn
		print('diff:', err, 'on', nn, 'notes', "{:.2f}".format(pc), '%')
		if (ks_gt == ks):
			table.append([name, '',  ks, nn, err, pc])
		else:
			table.append([name, ks_gt,  ks, nn, err, pc])
	df = pandas.DataFrame(table)
	df.columns = ['name', 'KSgt', 'KSest', 'notes', 'err', 'success']
    # every KSestimated identical to corresp. KSgt becomes NaN
	df.loc[df['KSgt'] == '', 'KSgt'] = np.nan
	# line total
	df.at['total', 'notes'] = df['notes'].sum()
	df.at['total', 'err'] = df['err'].sum()
	df.at['total', 'KSgt'] = len(table)
	df.at['total', 'KSest'] = df['KSgt'].isna().sum() # correct extimations of KS
	# line success rate
	nbn = df.at['total', 'notes']
	assert(nbn > 0)
	df.at['rate', 'success'] = (nbn - df.at['total', 'err'])*100/nbn
	assert(len(table) > 0)
	df.at['rate', 'KSest'] = df.at['total', 'KSest']*100/len(table)
	#df.at['rate', 'KSest'] = df.at['rate', 'KSest'].map('{:,.2f}'.format) 
	#df.at['percent', 'KSest'].apply('{:,.2f}'.format) 
	df['success'] = df['success'].map('{:,.2f}'.format) 
	df = df.convert_dtypes()
	output = 'csflag1' if csflag == 1 else 'csflag0' if csflag == 0 else 'default'
	#df.fillna('').to_csv(output+'.csv', header=True, index=False)
	df.to_csv(output+'.csv', header=True, index=False)
	return df

def FRB():
	# flag: flat corpus
	skip = ['Freddie the freeloader']
	return (get_corpus(Path('../../../Datasets')/'FakeRealBook'/'leads', True), skip)

def omnibook():
	# flag: flat corpus
	return get_corpus(Path('../../../Datasets')/'CharlieParkerOmnibook'/'musescore', True)  

def FiloSax(transposed=False):
	if transposed: # flag: flat corpus
		return get_corpus(Path('../../../Datasets')/'FiloSax-xml'/'transpose_ms', True)  
	else:	
		return get_corpus(Path('../../../Datasets')/'FiloSax-xml', True)  

def FiloBass():		
	return get_corpus(Path('../../../Datasets')/'FiloBass-xml', True)  

Using cpu


In [ ]:
filobass = FiloBass()

In [ ]:
eval_corpus(dataset=filobass, csflag=1)


 evaluation of All-the-Things-You-Are
estimated KS: -4 ground truth: -3
diff: 60 on 1056 notes 94.32 %

 evaluation of Alone-Together
estimated KS: -1 ground truth: -1
diff: 17 on 1087 notes 98.44 %

 evaluation of Apple-Jump
estimated KS: -1 ground truth: -1
diff: 70 on 1493 notes 95.31 %

 evaluation of Autumn-Leaves
estimated KS: -2 ground truth: -2
diff: 28 on 1055 notes 97.35 %

 evaluation of Blues-By-Five
estimated KS: -2 ground truth: -1
diff: 31 on 1003 notes 96.91 %

 evaluation of Bye-Bye-Blackbird
estimated KS: -1 ground truth: -1
diff: 30 on 1226 notes 97.55 %

 evaluation of C-Jam-Blues
estimated KS: 0 ground truth: 0
diff: 12 on 945 notes 98.73 %

 evaluation of Come-Rain-or-Come-Shine
estimated KS: -1 ground truth: -1
diff: 68 on 1396 notes 95.13 %

 evaluation of Cottontail
estimated KS: -4 ground truth: -2
diff: 31 on 2005 notes 98.45 %

 evaluation of Dear-Old-Stockholm
estimated KS: -1 ground truth: -1
diff: 10 on 1255 notes 99.20 %

 evaluation of Dolphin-dance
es

,name,KSgt,KSest,notes,err,success
0,All-the-Things-You-Are,-3,-4.0,1056,60,94.32
1,Alone-Together,<NA>,-1.0,1087,17,98.44
2,Apple-Jump,<NA>,-1.0,1493,70,95.31
3,Autumn-Leaves,<NA>,-2.0,1055,28,97.35
4,Blues-By-Five,-1,-2.0,1003,31,96.91
5,Bye-Bye-Blackbird,<NA>,-1.0,1226,30,97.55
6,C-Jam-Blues,<NA>,0.0,945,12,98.73
7,Come-Rain-or-Come-Shine,<NA>,-1.0,1396,68,95.13
8,Cottontail,-2,-4.0,2005,31,98.45
9,Dear-Old-Stockholm,<NA>,-1.0,1255,10,99.20
